# Hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 5
n_trials = 100
#first round of optimization
lgb_params = {"bagging_fraction": 0.9522534844058304, 
              "boosting_type": "dart", 
              "objective": "regression",
              "feature_fraction": 0.42236910941558053, 
              "lambda_l1": 0.020847266580277746, 
              "lambda_l2": 2.8448564854773326, 
              "learning_rate": 0.11484015430016059, 
              "max_depth": 3, 
              "max_leaves": 35, 
              "min_data_in_leaf": 9,
              "num_iterations": 150
             }
debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.selection import ShapSelector
from yspecies.tuning import Tune
from yspecies.models import ResultsCV, CrossValidator
from yspecies.results import FeatureSummary
import optuna
from optuna import Study, Trial

In [5]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [6]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [7]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 12323)",12323,38,408,"(12323, 2)","(38, 19)"


## Setting up ShapSelector ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [8]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2, [],  42)

## Setting up features to select ##

In [9]:
selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type = "split",
    clean_y_na = True
)

In [10]:
select_lifespan = replace(selection, select_by = "shap")

In [11]:
url = f'sqlite:///' +str((locations.metrics.lifespan / "study.sqlite").absolute())
print('loading (if exists) study from '+url)
storage = optuna.storages.RDBStorage(
    url=url
    #engine_kwargs={'check_same_thread': False}
)

study = optuna.multi_objective.study.create_study(directions=['minimize','maximize'], storage = storage)

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/metrics/lifespan/study.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-18 09:41:40,316] A new study created with name: no-name-55af020f-0ee9-4a29-9bb7-d9270080e660
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.


In [12]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 4.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 4.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [13]:
from yspecies.workflow import SplitReduce

def side(i: int):
    print(i)
    return i

prepare_partition = SplitReduce(
    outputs = DataPartitioner(), 
    split = lambda x: [(x[0], replace(partition_params, seed=side(x[2])))], 
    reduce = lambda x, output: (output[0], x[1]) 
)                               
partition_and_cv = Pipeline(
    [
        ("prepare partition", prepare_partition),
        ("shap_computation", ShapSelector()) #('crossvalidator', CrossValidator())        
    ]
)

partition_and_cv_repeat =  Pipeline([
    ("repeat_cv_pipe", Repeat(partition_and_cv, repeats, lambda x, i: [x[0], x[1], i] )),
    #("collect_mean", Collect(fold=lambda results: np.array([r.last("huber") for r in results]).mean()))
    ("collect_mean", Collect(fold=lambda results: (FeatureSummary(results).metrics_average.huber, FeatureSummary(results).kendall_tau_abs_mean)))    
    ]
    )

p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, study = study, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [ ]:
best = p.fit_transform((data, select_lifespan))
best

MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


0
fitting models with seed 0


Early stopping is not available in dart mode


[150]	valid_0's l1: 30.2068	valid_0's huber: 26.8229	valid_0's l2: 2808.43
[150]	valid_0's l1: 9.46266	valid_0's huber: 8.14054	valid_0's l2: 206.156
[150]	valid_0's l1: 7.94474	valid_0's huber: 6.76104	valid_0's l2: 156.122
[150]	valid_0's l1: 9.83255	valid_0's huber: 8.4809	valid_0's l2: 230.217
1
fitting models with seed 1


Early stopping is not available in dart mode


...

[150]	valid_0's l1: 8.89574	valid_0's huber: 7.64098	valid_0's l2: 256.184
[150]	valid_0's l1: 2.68065	valid_0's huber: 2.07251	valid_0's l2: 17.4621
[150]	valid_0's l1: 3.86839	valid_0's huber: 3.13108	valid_0's l2: 39.9152
[150]	valid_0's l1: 3.78379	valid_0's huber: 3.07594	valid_0's l2: 70.4408


MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-18 09:59:10,867] Trial 4 finished with values: [6.639873961824778, 0.48247706046810246] with parameters: {'boosting_type': 'dart', 'lambda_l1': 3.8735440936332064, 'lambda_l2': 2.4260607436541313, 'max_leaves': 25, 'max_depth': 5, 'feature_fraction': 0.6807050914694561, 'bagging_fraction': 0.9371686472950511, 'learning_rate': 0.08397181586317445, 'min_data_in_leaf': 7, 'drop_rate': 0.26053343108144117}.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The 

0
fitting models with seed 0
Training until validation scores don't improve for 10 rounds
[150]	valid_0's l1: 22.8054	valid_0's huber: 20.2293	valid_0's l2: 2219.29
Did not meet early stopping. Best iteration is:
[150]	valid_0's l1: 22.8054	valid_0's huber: 20.2293	valid_0's l2: 2219.29
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[107]	valid_0's l1: 3.59019	valid_0's huber: 2.9334	valid_0's l2: 44.786
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[79]	valid_0's l1: 2.71284	valid_0's huber: 2.14803	valid_0's l2: 25.7676
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[117]	valid_0's l1: 2.86751	valid_0's huber: 2.25271	valid_0's l2: 21.1949
1
fitting models with seed 1
Training until validation scores don't improve for 10 rounds
[150]	valid_0's l1: 2.88424	valid_0's huber: 2.28893	valid_0's l2: 28.4849
Did not meet early stopping. Best ite

MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-18 10:02:10,070] Trial 5 finished with values: [5.939046316036036, 0.47196198092417374] with parameters: {'boosting_type': 'gbdt', 'lambda_l1': 3.3743504637994306, 'lambda_l2': 0.7704411776915655, 'max_leaves': 25, 'max_depth': 8, 'feature_fraction': 0.4668868003340635, 'bagging_fraction': 0.9468230316409971, 'learning_rate': 0.06347384582562152, 'min_data_in_leaf': 7, 'drop_rate': 0.14756082600608192}.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The 

0
fitting models with seed 0
Training until validation scores don't improve for 10 rounds
[150]	valid_0's l1: 22.824	valid_0's huber: 20.252	valid_0's l2: 2241.49
Did not meet early stopping. Best iteration is:
[149]	valid_0's l1: 22.8239	valid_0's huber: 20.252	valid_0's l2: 2241.48
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[39]	valid_0's l1: 3.4635	valid_0's huber: 2.76063	valid_0's l2: 40.4425
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[65]	valid_0's l1: 2.71277	valid_0's huber: 2.14455	valid_0's l2: 24.951
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[62]	valid_0's l1: 3.04307	valid_0's huber: 2.42484	valid_0's l2: 24.5961
1
fitting models with seed 1
Training until validation scores don't improve for 10 rounds
[150]	valid_0's l1: 2.61057	valid_0's huber: 2.04094	valid_0's l2: 22.9868
Did not meet early stopping. Best iteratio

In [ ]:
print(best)

In [ ]:
print(best.params)

In [ ]:
print(best.results)